# Poisson discrete count model for MTurk demographics

In [2]:
%matplotlib inline
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

df = pd.DataFrame.from_csv("mturk_surveys.csv",index_col=None)
df.gender[df.gender=='Female']='female'
df

,Unnamed: 0,gender,hit_answered_date,hit_creation_date,household_income,household_size,location_city,location_country,location_region,marital_status,post_to_completion_secs,worker_id,year_of_birth
0,0,female,2016-09-14 17:37:11.297,2016-09-14 17:35:14.000,"$40,000-$59,999",1,mount holly,US,nj,divorced,117.0,a013d8ba79229f477b5b045bf35c7d57,1956
1,1,male,2016-09-14 17:29:09.539,2016-09-14 17:20:14.000,"Less than $10,000",2,pune,IN,mh,single,535.0,4958cd4059bd09a144c2698d50f5bc60,1992
2,2,female,2016-09-14 17:13:35.428,2016-09-14 16:20:14.000,"Less than $10,000",4,caracas,VE,a,divorced,3201.0,0a79a3e3ea28284e7a92ce0ddaa05e06,1978
3,3,female,2016-09-14 17:10:54.723,2016-09-14 17:05:13.000,"$75,000-$99,999",3,scottsdale,US,az,cohabitating,341.0,1079d94e2374ade52af1de09232c9797,1963
4,4,male,2016-09-14 16:52:20.792,2016-09-14 16:50:13.000,"Less than $10,000",4,ambala,IN,hr,single,127.0,897b71b78803b47bc19d0d31ff91856e,1993
5,5,female,2016-09-14 16:36:15.158,2016-09-14 16:35:14.000,"$75,000-$99,999",4,baldwinsville,US,ny,married,61.0,7a50b2a5e137a81bd5b5bb3c1bffc253,1970
6,6,male,2016-09-14 16:19:02.647,2016-09-14 15:35:16.000,"$10,000-$14,999",1,bangalore,IN,ka,single,2626.0,5e36b5553f8111cfafb56859641f4481,1991
7,7,female,2016-09-14 16:05:55.329,2016-09-14 16:05:16.000,"$25,000-$39,999",1,cambridge,US,ma,single,39.0,3ee130473a68a810fa64957e36500330,1982
8,8,female,2016-09-14 15:52:44.353,2016-09-14 15:50:14.000,"$60,000-$74,999",2,oklahoma city,US,ok,cohabitating,150.0,8457e3e06386f0f11fcdbba837d774bb,1991
9,9,female,2016-09-14 15:21:50.701,2016-09-14 15:20:16.000,"$75,000-$99,999",4,kansas city,US,mo,married,94.0,ad2bac11044e812a71adad3e29da27dd,1978


In [3]:
# By grouping by the informations, count the number of times each user working on the MTurk

df1 = df.groupby(['worker_id','gender','household_income','marital_status','household_size',
                  'location_country','year_of_birth']).size().sort_values()
df1=df1.reset_index()
df1.columns= ['worker_id','gender','household_income','marital_status','household_size',
              'location_country','year_of_birth','number_of_times']
df1

,worker_id,gender,household_income,marital_status,household_size,location_country,year_of_birth,number_of_times
0,0000ceeb6c91d4a1a53fec0de0cc3460,female,"$25,000-$39,999",single,1,US,1987,1
1,a14593ac153cc79be3b39dd5d7c92b10,male,"Less than $10,000",married,3,IN,1987,1
2,a14301a628ff22093311bd67a3b4377e,male,"$40,000-$59,999",married,2,DE,1982,1
3,a14301a628ff22093311bd67a3b4377e,male,"$40,000-$59,999",cohabitating,2,DE,1981,1
4,a14301a628ff22093311bd67a3b4377e,male,"$100,000 or more",married,3,DE,1980,1
5,a140dc952c3bd4a7d29738b21397c15a,female,"$25,000-$39,999",married,3,US,1977,1
6,a13f23143af6752d554b79d2e97a48a0,male,"Less than $10,000",single,3,ZZ,1988,1
7,a13f23143af6752d554b79d2e97a48a0,male,"Less than $10,000",single,3,IN,1988,1
8,a13f23143af6752d554b79d2e97a48a0,male,"$40,000-$59,999",single,4,IN,1989,1
9,a13f23143af6752d554b79d2e97a48a0,male,"$10,000-$14,999",single,3,IN,1987,1


In [7]:
df2 = df.groupby(['worker_id']).size().sort_values()
df2=df2.reset_index()
df2

,worker_id,0
0,0000ceeb6c91d4a1a53fec0de0cc3460,1
1,9adfe96d7d8f374c4ac8a0e0fc0f14cb,1
2,9ade47ff5fafd9ce469dc2f271244002,1
3,9ad8023da8e801f4a281c8d6d144634b,1
4,9ad736135fb545bf62ef87574b855808,1
5,9ad6721113a01e33d7077b19100da485,1
6,9ad5717127524806902e74bfbaa1958f,1
7,9ad503b874791c2f127ad704ec849fd3,1
8,9ad1d01b1075ee5ae9f82a252e6326b6,1
9,9acc6185ab15a4d4522605b738d620fe,1


In [18]:
df1[df1.worker_id=='cc15e2c74af011007361cef2b44fd33f']

,index,worker_id,gender,household_income,marital_status,household_size,location_country,year_of_birth,number_of_times
15567,15567,cc15e2c74af011007361cef2b44fd33f,male,"$25,000-$39,999",single,3,US,1971,1
15568,15568,cc15e2c74af011007361cef2b44fd33f,male,"$15,000-$24,999",single,3,US,1989,1
35606,35606,cc15e2c74af011007361cef2b44fd33f,male,"$25,000-$39,999",single,3,US,1989,5
36444,36444,cc15e2c74af011007361cef2b44fd33f,male,"$25,000-$39,999",single,3,US,1970,13
